# Install dependencies


In [1]:
!pip install -qU torch==2.5.1 torchvision==0.20.1 "ray[client,train]"==2.39.0

# Import dependencies


In [2]:
import os
from typing import Dict

import torch
from filelock import FileLock
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.transforms import Normalize, ToTensor

import ray
import ray.train
from ray.train import ScalingConfig
from ray.train.torch import TorchTrainer

# Define functions


In [3]:
def get_dataloaders(batch_size):
    # Transform to normalize the input images
    transform = transforms.Compose([ToTensor(), Normalize((0.5,), (0.5,))])

    with FileLock(os.path.expanduser("~/data.lock")):
        # Download training data from open datasets
        training_data = datasets.FashionMNIST(
            root="~/data",
            train=True,
            download=True,
            transform=transform,
        )

        # Download test data from open datasets
        test_data = datasets.FashionMNIST(
            root="~/data",
            train=False,
            download=True,
            transform=transform,
        )

    # Create data loaders
    train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
    test_dataloader = DataLoader(test_data, batch_size=batch_size)

    return train_dataloader, test_dataloader


# Model Definition
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(512, 10),
            nn.ReLU(),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


def train_func_per_worker(config: Dict):
    lr = config["lr"]
    epochs = config["epochs"]
    batch_size = config["batch_size_per_worker"]

    # Get dataloaders inside the worker training function
    train_dataloader, test_dataloader = get_dataloaders(batch_size=batch_size)

    # [1] Prepare Dataloader for distributed training
    # Shard the datasets among workers and move batches to the correct device
    # =======================================================================
    train_dataloader = ray.train.torch.prepare_data_loader(train_dataloader)
    test_dataloader = ray.train.torch.prepare_data_loader(test_dataloader)

    model = NeuralNetwork()

    # [2] Prepare and wrap your model with DistributedDataParallel
    # Move the model to the correct GPU/CPU device
    # ============================================================
    model = ray.train.torch.prepare_model(model)

    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9)

    # Model training loop
    for epoch in range(epochs):
        if ray.train.get_context().get_world_size() > 1:
            # Required for the distributed sampler to shuffle properly across epochs.
            train_dataloader.sampler.set_epoch(epoch)

        model.train()
        print(f"Train Epoch {epoch}")
        for X, y in train_dataloader:
            pred = model(X)
            loss = loss_fn(pred, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        model.eval()
        test_loss, num_correct, num_total = 0, 0, 0
        with torch.no_grad():
            print(f"Test Epoch {epoch}")
            for X, y in test_dataloader:
                pred = model(X)
                loss = loss_fn(pred, y)

                test_loss += loss.item()
                num_total += y.shape[0]
                num_correct += (pred.argmax(1) == y).sum().item()

        test_loss /= len(test_dataloader)
        accuracy = num_correct / num_total

        # [3] Report metrics to Ray Train
        # ===============================
        ray.train.report(metrics={"loss": test_loss, "accuracy": accuracy})


@ray.remote
def train_fashion_mnist(num_workers=2, use_gpu=False):
    global_batch_size = 32

    train_config = {
        "lr": 1e-3,
        "epochs": 10,
        "batch_size_per_worker": global_batch_size // num_workers,
    }

    # Configure computation resources
    scaling_config = ScalingConfig(num_workers=num_workers, use_gpu=use_gpu)

    # Initialize a Ray TorchTrainer
    trainer = TorchTrainer(
        train_loop_per_worker=train_func_per_worker,
        train_loop_config=train_config,
        scaling_config=scaling_config,
    )

    # [4] Start distributed training
    # Run `train_func_per_worker` on all workers
    # =============================================
    result = trainer.fit()
    print(f"Training result: {result}")

# Connect to Ray Cluster


In [4]:
ray.init(
    address="ray://raycluster-head-svc.default.svc.cluster.local:10001",
    runtime_env={"pip": ["torch==2.5.1", "torchvision==0.20.1"]},
)

2024-11-26 07:46:18,158	INFO client_builder.py:244 -- Passing the following kwargs to ray.init() on the server: log_to_driver
2024-11-26 07:48:18,245	WARNING utils.py:1591 -- Python patch version mismatch: The cluster was started with:
    Ray: 2.39.0
    Python: 3.9.20
This process on Ray Client was started with:
    Ray: 2.39.0
    Python: 3.9.13



Python version:,3.9.20
Ray version:,2.39.0
Dashboard:,http://10.0.2.162:8265


# Train model remotely


In [5]:
ray.get(train_fashion_mnist.remote(num_workers=7, use_gpu=False))

(autoscaler +10s) Tip: use `ray status` to view detailed cluster status. To disable these messages, set RAY_SCHEDULER_EVENTS=0.
(autoscaler +10s) Adding 1 node(s) of type cpu-group.
(autoscaler +10s) Resized to 7 CPUs.
(train_fashion_mnist pid=483, ip=10.0.2.96) 
(train_fashion_mnist pid=483, ip=10.0.2.96) View detailed results here: /home/ray/ray_results/TorchTrainer_2024-11-25_23-51-36
(train_fashion_mnist pid=483, ip=10.0.2.96) To visualize your results with TensorBoard, run: `tensorboard --logdir /tmp/ray/session_2024-11-25_23-44-13_149600_1/artifacts/2024-11-25_23-51-36/TorchTrainer_2024-11-25_23-51-36/driver_artifacts`
(autoscaler +3m23s) Adding 1 node(s) of type cpu-group.
(autoscaler +3m23s) Resized to 14 CPUs.
(autoscaler +3m23s) No available node types can fulfill resource requests {'bundle_group_f03e72a62d6a797333cee28989ec01000000': 0.001}*1. Add suitable node types to this cluster to resolve this issue.
(autoscaler +3m28s) No available node types can fulfill resource reque

(RayTrainWorker pid=763, ip=10.0.2.96) Setting up process group for: env:// [rank=0, world_size=7]
(TorchTrainer pid=685, ip=10.0.2.96) Started distributed worker processes: 
(TorchTrainer pid=685, ip=10.0.2.96) - (node_id=a9e7f8e16f54268c4de964e5e5d30ca1e5ace07cd1453e6d4a1fd681, ip=10.0.2.96, pid=763) world_rank=0, local_rank=0, node_rank=0
(TorchTrainer pid=685, ip=10.0.2.96) - (node_id=a9e7f8e16f54268c4de964e5e5d30ca1e5ace07cd1453e6d4a1fd681, ip=10.0.2.96, pid=764) world_rank=1, local_rank=1, node_rank=0
(TorchTrainer pid=685, ip=10.0.2.96) - (node_id=a9e7f8e16f54268c4de964e5e5d30ca1e5ace07cd1453e6d4a1fd681, ip=10.0.2.96, pid=766) world_rank=2, local_rank=2, node_rank=0
(TorchTrainer pid=685, ip=10.0.2.96) - (node_id=a9e7f8e16f54268c4de964e5e5d30ca1e5ace07cd1453e6d4a1fd681, ip=10.0.2.96, pid=767) world_rank=3, local_rank=3, node_rank=0
(TorchTrainer pid=685, ip=10.0.2.96) - (node_id=a9e7f8e16f54268c4de964e5e5d30ca1e5ace07cd1453e6d4a1fd681, ip=10.0.2.96, pid=768) world_rank=4, local_

(RayTrainWorker pid=766, ip=10.0.2.96) Downloading http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-images-idx3-ubyte.gz
(RayTrainWorker pid=467, ip=10.0.2.69) Downloading http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-images-idx3-ubyte.gz
(RayTrainWorker pid=766, ip=10.0.2.96) Downloading http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-images-idx3-ubyte.gz to /home/ray/data/FashionMNIST/raw/train-images-idx3-ubyte.gz
(RayTrainWorker pid=467, ip=10.0.2.69) Downloading http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-images-idx3-ubyte.gz to /home/ray/data/FashionMNIST/raw/train-images-idx3-ubyte.gz


  0%|          | 0.00/26.4M [00:00<?, ?B/s] 
  0%|          | 0.00/26.4M [00:00<?, ?B/s] 
  0%|          | 32.8k/26.4M [00:00<02:06, 208kB/s]
  0%|          | 65.5k/26.4M [00:00<02:07, 207kB/s]
  0%|          | 65.5k/26.4M [00:00<01:27, 302kB/s]
  0%|          | 131k/26.4M [00:00<01:27, 301kB/s] 
  0%|          | 131k/26.4M [00:00<01:13, 359kB/s] 
  1%|          | 229k/26.4M [00:00<01:01, 427kB/s]
  1%|          | 197k/26.4M [00:00<01:08, 382kB/s]
  2%|▏         | 459k/26.4M [00:00<00:32, 795kB/s]
  1%|          | 262k/26.4M [00:00<01:06, 394kB/s]
  3%|▎         | 918k/26.4M [00:00<00:15, 1.69MB/s]
  7%|▋         | 1.84M/26.4M [00:01<00:06, 3.54MB/s]
  1%|▏         | 393k/26.4M [00:00<00:47, 546kB/s]
  9%|▊         | 2.26M/26.4M [00:01<00:06, 3.70MB/s]
  3%|▎         | 688k/26.4M [00:01<00:26, 984kB/s]
 14%|█▍        | 3.70M/26.4M [00:01<00:03, 5.72MB/s]
  5%|▍         | 1.31M/26.4M [00:01<00:12, 1.93MB/s]
 25%|██▍       | 6.59M/26.4M [00:01<00:01, 10.0MB/s]
 10%|▉         | 2.56M/26.4

(RayTrainWorker pid=467, ip=10.0.2.69) Extracting /home/ray/data/FashionMNIST/raw/train-images-idx3-ubyte.gz to /home/ray/data/FashionMNIST/raw


...

(RayTrainWorker pid=766, ip=10.0.2.96) Extracting /home/ray/data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to /home/ray/data/FashionMNIST/raw
(RayTrainWorker pid=766, ip=10.0.2.96) 


100%|██████████| 5.15k/5.15k [00:00<00:00, 45.9MB/s]
(RayTrainWorker pid=763, ip=10.0.2.96) Moving model to device: cpu
(RayTrainWorker pid=763, ip=10.0.2.96) Wrapping provided model in DistributedDataParallel.


(RayTrainWorker pid=467, ip=10.0.2.69) Train Epoch 0
(RayTrainWorker pid=763, ip=10.0.2.96) Train Epoch 0
(RayTrainWorker pid=764, ip=10.0.2.96) Train Epoch 0
(RayTrainWorker pid=766, ip=10.0.2.96) Train Epoch 0
(RayTrainWorker pid=767, ip=10.0.2.96) Train Epoch 0
(RayTrainWorker pid=768, ip=10.0.2.96) Train Epoch 0
(RayTrainWorker pid=765, ip=10.0.2.96) Train Epoch 0
(RayTrainWorker pid=467, ip=10.0.2.69) Test Epoch 0
(RayTrainWorker pid=763, ip=10.0.2.96) Test Epoch 0
(RayTrainWorker pid=764, ip=10.0.2.96) Test Epoch 0
(RayTrainWorker pid=766, ip=10.0.2.96) Test Epoch 0
(RayTrainWorker pid=767, ip=10.0.2.96) Test Epoch 0
(RayTrainWorker pid=768, ip=10.0.2.96) Test Epoch 0
(RayTrainWorker pid=765, ip=10.0.2.96) Test Epoch 0
(RayTrainWorker pid=467, ip=10.0.2.69) Train Epoch 1
(train_fashion_mnist pid=483, ip=10.0.2.96) 
(train_fashion_mnist pid=483, ip=10.0.2.96) Training finished iteration 1 at 2024-11-25 23:54:02. Total running time: 2min 26s
(train_fashion_mnist pid=483, ip=10.0.2.

(train_fashion_mnist pid=483, ip=10.0.2.96) Wrote the latest version of all result files and experiment state to '/home/ray/ray_results/TorchTrainer_2024-11-25_23-51-36' in 0.0028s.


(train_fashion_mnist pid=483, ip=10.0.2.96) 
(train_fashion_mnist pid=483, ip=10.0.2.96) Training completed after 10 iterations at 2024-11-25 23:58:12. Total running time: 6min 36s
(train_fashion_mnist pid=483, ip=10.0.2.96) 
(train_fashion_mnist pid=483, ip=10.0.2.96) Training result: Result(
(train_fashion_mnist pid=483, ip=10.0.2.96)   metrics={'loss': 0.35352502386181617, 'accuracy': 0.8726382085374388},
(train_fashion_mnist pid=483, ip=10.0.2.96)   path='/home/ray/ray_results/TorchTrainer_2024-11-25_23-51-36/TorchTrainer_4343d_00000_0_2024-11-25_23-51-36',
(train_fashion_mnist pid=483, ip=10.0.2.96)   filesystem='local',
(train_fashion_mnist pid=483, ip=10.0.2.96)   checkpoint=None
(train_fashion_mnist pid=483, ip=10.0.2.96) )


# Shut down Ray workers


In [6]:
ray.shutdown()